# GPU Utilization Analysis

In [9]:
import os

import torch
import wandb
from tqdm import tqdm
import seqdata as sd

import eugene as eu
import eugene.train
import eugene.models
from eugene import settings
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/kopp21"
settings.output_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/output/kopp21"
settings.logging_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/kopp21"
settings.config_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/configs/kopp21"

In [2]:
wandb.login()

wandb: Currently logged in as: dlaub. Use `wandb login --relogin` to force relogin


True

## Load a medium sized example dataset

In [3]:
sdata = sd.open_zarr(os.path.join(settings.dataset_dir, 'kopp21_train.zarr'))
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 948771, cov_sample: 1, _length: 500, length: 500,
                 _ohe: 4)
Coordinates:
  * cov_sample  (cov_sample) object 'stam'
Dimensions without coordinates: _sequence, _length, length, _ohe
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(59299,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(59299,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(59299,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(29650, 1, 32), meta=np.ndarray>
    ohe_seq     (_sequence, length, _ohe) uint8 dask.array<chunksize=(59299, 63, 1), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(59299, 32), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(59299,), meta=np.ndarray>
    target      (_sequence) uint8 dask.array<chunksize=(237193,), meta=np.ndarray>
    train_test  (_sequence) bool dask.array<chunksize=(237193,), meta=np.ndarray>
    train_val   (_sequence) bool dask.array<chunksize=(237193,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

In [4]:
sdata[['ohe_seq', 'target']].load();

In [5]:
transforms = {
    "ohe_seq": lambda x: x.swapaxes(1, 2)
}

In [13]:
train_sdata = sdata.sel(_sequence=(sdata.train_val == True).compute())
dloader = sd.get_torch_dataloader(
    train_sdata,
    sample_dims='_sequence',
    variables=['ohe_seq', 'target'],
    transforms=transforms,
    batch_size=2048,
    pin_memory=True,
    drop_last=False,
)

## Build or choose a medium sized model

In [6]:
def prep_new_model(
    config,
    seed,
):
    # Instantiate the model
    model = eu.models.load_config(
        config_path=config,
        seed=seed
    )

    # Initialize the model prior to conv filter initialization
    eu.models.init_weights(model)

    # Return the model
    return model

In [7]:
kopp21 = prep_new_model("kopp21_cnn.yaml", seed=0).to('cuda')

[rank: 0] Global seed set to 0


## Use native PyTorch to train the model and record GPU utilization

In [90]:
def train(model, dloader, optim, loss_fn):
    model.train()
    for batch in tqdm(dloader, leave=False):
        pred = model(batch['ohe_seq'].to('cuda'))
        loss = loss_fn(pred.squeeze(), batch['target'].to('cuda'))
        loss.backward()
        optim.step()
        optim.zero_grad()

In [92]:
optim = torch.optim.Adam(kopp21.arch.parameters())

with wandb.init(project='EUGENe GPU Utilization', name='Native PyTorch'):
    epochs = 25
    for _ in tqdm(range(epochs)):
        train(kopp21.arch, dloader, optim, kopp21.loss_fxn)

100%|██████████| 25/25 [06:37<00:00, 15.92s/it]


## Use PyTorch Lightning to train the model and record GPU utilization

In [42]:
import xarray as xr
import pytorch_lightning as pl

In [15]:
batch_size = 2048
num_workers = 0
target_keys = ['target']
seq_key = 'ohe_seq'
in_memory = True
train_key = 'train_val'
prefetch_factor = None
drop_last = False

if target_keys is not None:
    if isinstance(target_keys, str):
        target_keys = [target_keys]
    if len(target_keys) == 1:
        sdata["target"] = sdata[target_keys[0]]
    else:
        sdata["target"] = xr.concat(
            [sdata[target_key] for target_key in target_keys], dim="_targets"
        ).transpose("_sequence", "_targets")
    nan_mask = sdata['target'].isnull()
    if sdata["target"].ndim > 1:
        nan_mask = nan_mask.any('_targets')
    print(f"Dropping {nan_mask.sum().compute().item()} sequences with NaN targets.")
if in_memory:
    print(f"Loading {seq_key} and {target_keys} into memory")
    sdata[seq_key].load()
    sdata["target"].load()

Dropping 0 sequences with NaN targets.
Loading ohe_seq and ['target'] into memory


In [17]:
train_sdata = sdata.sel(_sequence=(sdata[train_key] == True).compute())
val_sdata = sdata.sel(_sequence=(sdata[train_key] == False).compute())

In [47]:
train_dataloader = sd.get_torch_dataloader(
    train_sdata,
    sample_dims=["_sequence"],
    variables=['ohe_seq', "target"],
    batch_size=batch_size,
    num_workers=num_workers,
    prefetch_factor=prefetch_factor,
    transforms=transforms,
    shuffle=True,
    drop_last=drop_last,
)

Get train dl


In [44]:
val_dataloader = sd.get_torch_dataloader(
    val_sdata,
    sample_dims=["_sequence"],
    variables=['ohe_seq', "target"],
    batch_size=batch_size,
    num_workers=num_workers,
    prefetch_factor=prefetch_factor,
    transforms=transforms,
    shuffle=False,
    drop_last=drop_last,
)

In [49]:
trainer = pl.Trainer(
    max_epochs=25,
    logger=False,
    devices='auto',
    accelerator="gpu",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [53]:
kopp21.scheduler = None

In [54]:
kopp21 = prep_new_model("kopp21_cnn.yaml", seed=0).to('cuda')
with wandb.init(project='EUGENe GPU Utilization', name='PL no val'):
    trainer.fit(kopp21, train_dataloaders=train_dataloader)

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/notebooks/gpu_utilization_analysis/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and 

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


## Use EUGENe to train the model and record GPU utilization

EUGENe requires a validation set to evaluate on at every epoch and conditionally reduce the learning rate.

In [10]:
kopp21 = prep_new_model("kopp21_cnn.yaml", seed=0).to('cuda')
with wandb.init(project='EUGENe GPU Utilization', name='EUGENe'):
    eu.train.fit_sequence_module(
        kopp21,
        sdata,
        gpus=1,
        seq_key="ohe_seq",
        target_keys=["target"],
        in_memory=True,
        train_key="train_val",
        epochs=25,
        batch_size=2048,
        drop_last=False,
        transforms=transforms,
        early_stopping_metric=None,
        model_checkpoint_monitor=None,
    )

Dropping 0 sequences with NaN targets.
Loading ohe_seq and ['target'] into memory
No seed set


/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/kopp21/SequenceModule
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type        | Params
---------------------------------------------
0 | arch         | Kopp21CNN   | 743   
1 | train_metric | BinaryAUROC | 0     
2 | val_metric   | BinaryAUROC | 0     
3 | test_metric  | BinaryAUROC | 0     
---------------------------------------------
743       

Sanity Checking: 0it [00:00, ?it/s]

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottl

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.
